Download the datasets from this link: https://research.unsw.edu.au/projects/bot-iot-dataset

Testing: UNSW_2018_IoT_Botnet_Final_10_best_Training.csv

Training: UNSW_2018_IoT_Botnet_Final_10_best_Testing.csv

We are using the 10 attributes given from the 6th to 16th column in the data which represent aggregrate network information. The output is for now is only whether or not the network is part of a bot net: 1 for yes, 0 for no. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [3]:
df = pd.read_csv('UNSW_2018_IoT_Botnet_Final_10_best_Training.csv')

In [4]:
df2 = pd.read_csv('UNSW_2018_IoT_Botnet_Final_10_best_Testing.csv')

Pick out the attribute columns for training and testing.

In [5]:
X_train = df.iloc[:,6:16]
y_train = df.iloc[:,16:17]

In [6]:
y_train.head()

,attack
0,1
1,1
2,1
3,1
4,1


In [7]:
X_test = df2.iloc[:,6:16]
y_test = df2.iloc[:,16:17]

Training Decision Tree Model

In [8]:
DT_clf = DecisionTreeClassifier()
DT_clf = DT_clf.fit(X_train,y_train)

In [9]:
DT_y_pred = DT_clf.predict(X_test)

In [10]:
acc = accuracy_score(y_test, DT_y_pred)

In [11]:
print("Accuracy:", acc)
print("Classification Report:")
print(classification_report(y_test, DT_y_pred))

Accuracy: 0.9999890964352158
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.93      0.96       107
           1       1.00      1.00      1.00    733598

    accuracy                           1.00    733705
   macro avg       1.00      0.97      0.98    733705
weighted avg       1.00      1.00      1.00    733705



Training K-nearest neighbors model

In [12]:
from sklearn.neighbors import KNeighborsClassifier

In [13]:
KNN_clf = KNeighborsClassifier(n_neighbors=5)
KNN_clf = KNN_clf.fit(X_train,y_train.values.ravel()) #using ravel to change column vector to a 1-D array

In [14]:
KNN_y_pred = KNN_clf.predict(X_test)

In [15]:
KNN_acc = accuracy_score(y_test, KNN_y_pred)

In [16]:
print("Accuracy:", KNN_acc)
print("Classification Report:")
print(classification_report(y_test, KNN_y_pred))

Accuracy: 0.9999291268289027
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.55      0.69       107
           1       1.00      1.00      1.00    733598

    accuracy                           1.00    733705
   macro avg       0.97      0.78      0.85    733705
weighted avg       1.00      1.00      1.00    733705



Training Naive Bayes Model 

In [17]:
from sklearn.naive_bayes import GaussianNB

In [18]:
NB_clf = GaussianNB()
NB_clf = NB_clf.fit(X_train,y_train.values.ravel())  #using ravel to change column vector to a 1-D array

In [19]:
NB_y_pred = NB_clf.predict(X_test)

In [20]:
NB_acc = accuracy_score(y_test, NB_y_pred)

In [21]:
print("Accuracy:", NB_acc)
print("Classification Report:")
print(classification_report(y_test, NB_y_pred))

Accuracy: 0.9970165120859201
Classification Report:
              precision    recall  f1-score   support

           0       0.03      0.69      0.06       107
           1       1.00      1.00      1.00    733598

    accuracy                           1.00    733705
   macro avg       0.52      0.84      0.53    733705
weighted avg       1.00      1.00      1.00    733705



Signature-based Detection (Non-ML)

In [22]:

import re


signatures = [
    r'^.* ICMP .*$',                   # ICMP traffic
    r'^.* TCP .*\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:23\b.*$',   # Telnet traffic (port 23)
    r'^.* TCP .*\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:2323\b.*$', # Telnet traffic (port 2323)
    r'^.* TCP .*\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:5555\b.*$'  # Backdoor traffic (port 5555)
]
def match_signatures(traffic):
    for signature in signatures:
        if re.match(signature, traffic, re.IGNORECASE):
            return 1  # Botnet activity detected
    return 0  # No botnet activity detected

df['Signature_Detection'] = df['Activity'].apply(match_signatures)

# Evaluation and Comparison
true_labels = df['Label'].values
signature_labels = df['Signature_Detection'].values

signature_accuracy = (true_labels == signature_labels).mean()
print(f"Signature-based Detection Accuracy: {signature_accuracy:.4f}")

KeyError: 'Activity'